In [1]:
import json
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import os
from os import listdir
import re
import bisect
import random
import pickle
from ProactiveUtil import *

In [7]:
#the following is loading testing dataset
filepath = '../../data/iptvdata/'
# filename = 'iptv_sample1000000.csv'
filename = 'iptv_sample123987.csv'
df = pd.read_csv(filepath+filename)
df = df.sort_values(by=['time'],ascending = (True))
df

,time,user,item
15360,2010-06-24 00:00:11,00111010000100010001011101010011,触电情缘(第15集).mp4
24887,2010-06-24 00:00:42,00111010000100010011001110101111,新三国(第29集).mp4
48925,2010-06-24 00:00:58,00111010000100010110110110100111,G4特工.mp4
15361,2010-06-24 00:01:01,00111010000100010001011101010011,触电情缘(第15集).mp4
54855,2010-06-24 00:01:16,00111010000100010111010010000110,算死草.mp4
...,...,...,...
4084,2010-07-06 23:58:23,00111010000100010001001100110100,虎胆龙威4.mp4
24028,2010-07-06 23:58:45,00111010000100010011001001111000,黑执事(第06集).mp4
34829,2010-07-06 23:58:50,00111010000100010100101101011100,全职杀手.mp4
38871,2010-07-06 23:59:25,00111010000100010101001101001000,包三姑外传(第01集).mp4


In [8]:
#dataset parameter:
dataset_begintime = datetime(2010,6,24)
ratio = 0.5
df['time'] = pd.to_datetime(df['time'])
splittime = dataset_begintime + (df['time'].max()- dataset_begintime)*ratio
df_test = df[df['time']>splittime]
df_test

,time,user,item
49590,2010-06-30 11:59:53,00111010000100010110110111101100,变形金刚(蓝光).mp4
12965,2010-06-30 12:00:14,00111010000100010001011101010010,鬼妈妈.mp4
49591,2010-06-30 12:00:23,00111010000100010110110111101100,变形金刚(蓝光).mp4
49592,2010-06-30 12:00:25,00111010000100010110110111101100,蓝色激情.mp4
2043,2010-06-30 12:00:41,00111010000100010000101011001011,那些迷人的往事(第28集).mp4
...,...,...,...
4084,2010-07-06 23:58:23,00111010000100010001001100110100,虎胆龙威4.mp4
24028,2010-07-06 23:58:45,00111010000100010011001001111000,黑执事(第06集).mp4
34829,2010-07-06 23:58:50,00111010000100010100101101011100,全职杀手.mp4
38871,2010-07-06 23:59:25,00111010000100010101001101001000,包三姑外传(第01集).mp4


In [9]:
df_test.dropna(inplace=True)
# np.nan in list(['item'])
df_test

C:\Users\lilee\anaconda3new\envs\temporal_3_6python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,time,user,item
49590,2010-06-30 11:59:53,00111010000100010110110111101100,变形金刚(蓝光).mp4
12965,2010-06-30 12:00:14,00111010000100010001011101010010,鬼妈妈.mp4
49591,2010-06-30 12:00:23,00111010000100010110110111101100,变形金刚(蓝光).mp4
49592,2010-06-30 12:00:25,00111010000100010110110111101100,蓝色激情.mp4
2043,2010-06-30 12:00:41,00111010000100010000101011001011,那些迷人的往事(第28集).mp4
...,...,...,...
4084,2010-07-06 23:58:23,00111010000100010001001100110100,虎胆龙威4.mp4
24028,2010-07-06 23:58:45,00111010000100010011001001111000,黑执事(第06集).mp4
34829,2010-07-06 23:58:50,00111010000100010100101101011100,全职杀手.mp4
38871,2010-07-06 23:59:25,00111010000100010101001101001000,包三姑外传(第01集).mp4


In [10]:
def caching_simulation_lru(CACHE_SIZE = 500,N=1, downloadtime = 0.5,printoutfrequency = 10000,begintime = datetime(2010,7,5),stop_time = datetime(2010,7,7)):
#     ngram_dict = {}
#     ngram_dict_confidence = {}
    total_count = 0
    local_count = 0
    your_cache = set()
    hit = 0
    accum_hit = 0
    soft_hit = 0
    proactive_download = 0
    reactive_download_number =0
    hit_rate_list = []
    x_time_list = []
    req_rate_list = []
    utilization_list_reactive = []
    past_timestamps = {}
    reactive_latency = 0
    proactive_latency = 0
    calculatetime = begintime
    bandwidth_link = {'begin time':[],'end time':[],'item':[],'proactive_indicator':[]}
    print('cache size:',CACHE_SIZE, "N ", N,'downloadtime',downloadtime,'begintime',begintime,'stoptime',stop_time)
    for index, row in df_test.iterrows():
        user = row[1]
        time = row[0].to_pydatetime()
#         time = datetime(2010,6,24) + timedelta(seconds=time)
#         print(time)
    #         location = row[2]
        item = row[2]
#         if len(item.split('(第'))>=2:
#     #         print(item)
#             pass
#         else:
#             continue
        if time < begintime:
            continue
        #initial link empty time is the begin time
        if total_count == 0:
            link_empty_time = time

        total_count += 1
        local_count += 1
#-------------------------------------------------------------------------------------------------------------------
        #update link
        for i in range(len(bandwidth_link['item'])):
            if bandwidth_link['end time'][0] < time:
                req_time = bandwidth_link['begin time'].pop(0)
                downloaoded_time = bandwidth_link['end time'].pop(0)
                downloaded_item = bandwidth_link['item'].pop(0)
                proactive_indicator = bandwidth_link['proactive_indicator'].pop(0)

                if len(your_cache) < CACHE_SIZE:
                    #add item to cache
                    your_cache.add(downloaded_item)

                else:
                    #update cache

                    your_cache.add(downloaded_item)
                    position, kick_name, pasttimeforrank = get_lruK_action_rank(N, your_cache, past_timestamps,downloaoded_time)
                    your_cache.remove(kick_name)


                if proactive_indicator is False:
                    # for proactive latency, we calculate that in other part.
                    # get reactive latency
                    reactive_latency += (downloaoded_time - req_time).total_seconds()
                    # print((downloaoded_time - req_time).total_seconds())
                    if reactive_latency<0:
                        print('error')
#-------------------------------------------------------------------------------------------------------------------
        if item not in past_timestamps:
            past_timestamps[item] = [time]
        else:
            past_timestamps[item].append(time)

#-------------------------------------------------------------------------------------------------------------------
        if item in your_cache:
            hit += 1
            accum_hit += 1
            # deffect(time_window,time,user,name,scores)
        elif item in bandwidth_link['item']:
            soft_hit += 1
        else:
            # not hit and download
            bandwidth_link['begin time'].append(time)
            bandwidth_link['end time'].append(max(link_empty_time,time) + timedelta(seconds=downloadtime))
            bandwidth_link['item'].append(item)
            bandwidth_link['proactive_indicator'].append(False)
            reactive_download_number += 1
            link_empty_time = max(link_empty_time,time) + timedelta(seconds=downloadtime)
#-------------------------------------------------------------------------------------------------------------------

        if total_count % printoutfrequency == 0:
            timedelta_frequency = (time - calculatetime).total_seconds()

#             print('delta',timedelta_frequency)
            reactive_number_in_bandwidth = 0
            proactive_number_in_bandwidth = 0
            for i in range(len(bandwidth_link['item'])):
                if bandwidth_link['proactive_indicator'][i]:
                    proactive_number_in_bandwidth += 1
                else:
                    reactive_number_in_bandwidth += 1

#             print('reactive donwload number',reactive_download_number)
#             print('proactive donwload number',proactive_download)
#                 print('reactive in link',reactive_number_in_bandwidth)
#                 print('roactive in link',proactive_number_in_bandwidth)
            reactive_utilization = round((reactive_download_number - reactive_number_in_bandwidth)*downloadtime / timedelta_frequency,4)
            utilization_list_reactive.append(reactive_utilization)
#             print(reactive_utilization)
            
            
            
            hit_rate_list.append(hit/local_count)
            x_time_list.append(str(time.day) +'_'+ str(time.hour))
            req_rate = round(printoutfrequency/(time - calculatetime).total_seconds()*60,2)
            calculatetime = time
            req_rate_list.append(req_rate)
            print(round(hit / local_count, 4),'proactive download',proactive_download,soft_hit,'buffer content',len(bandwidth_link['item']))
            
            hit = 0
            local_count = 0
            proactive_download = 0
            reactive_download_number =0
            soft_hit = 0
            print(time)
#             print(req_rate)
            

#         if total_count == stop_point:
        if time >= stop_time:
            print('total ccount',total_count)
            print('hit ratio',hit_rate_list)
            print(round(accum_hit / total_count, 4), 'latency', proactive_latency + reactive_latency)
            print('x axis time',x_time_list)
            print('req rate',req_rate_list)
            print('utilization',utilization_list_reactive)
            # x = range(len(hit_rate_list))
            # plt.plot(x, hit_rate_list)
            # plt.title('LRU caching')
            # plt.legend()
            # plt.show()
#             print(total_count)

            return proactive_latency + reactive_latency, hit_rate_list


    print(total_count)
    print(hit_rate_list)
    print (round(accum_hit / total_count,4),'latency',proactive_latency+reactive_latency)
    # x = range(len(hit_rate_list))
    # plt.plot(x, hit_rate_list)
    # plt.title('LRU caching')
    # plt.legend()
    # plt.show()

    return proactive_latency+reactive_latency,hit_rate_list


In [11]:
# for size in [50,100,200,500,1000]:

# size = 2000
# for size in [500,1000,2000,5000]:
for size in [50,100,200,500,1000]:
    for downloadt in [0.5]:
        caching_simulation_lru(CACHE_SIZE = size,N=2,downloadtime = downloadt,printoutfrequency = 5000,begintime = datetime(2010,7,5),stop_time = datetime(2010,7,7))

cache size: 50 N  2 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
0.1134 proactive download 0 64 buffer content 1
2010-07-05 22:46:40
0.102 proactive download 0 55 buffer content 2
2010-07-06 20:32:08
11716
[0.1134, 0.102]
0.1059 latency 6152.5
cache size: 100 N  2 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
0.14 proactive download 0 63 buffer content 1
2010-07-05 22:46:40
0.1294 proactive download 0 54 buffer content 2
2010-07-06 20:32:08
11716
[0.14, 0.1294]
0.1326 latency 5978.0
cache size: 200 N  2 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
0.173 proactive download 0 61 buffer content 1
2010-07-05 22:46:40
0.1626 proactive download 0 53 buffer content 2
2010-07-06 20:32:08
11716
[0.173, 0.1626]
0.1658 latency 5752.5
cache size: 500 N  2 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
0.2414 proactive download 0 52 buffer content 1
2010-07-05 22:46:40
0.244 proa